## pandas-profiling Meteorites example
Source of data: https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh

### Import libraries

In [20]:
import pandas as pd
import dask.dataframe as dd
import dask_profiling
import numpy as np
import os

### Load and prepare example dataset
We add some fake variables for illustrating pandas-profiling capabilities

In [13]:
df = pd.read_csv("https://data.nasa.gov/api/views/gh4g-9sfh/rows.csv?accessType=DOWNLOAD", parse_dates=['year'], encoding='UTF-8')

# Note: Pandas does not support dates before 1880, so we ignore these for this analysis
df['year'] = pd.to_datetime(df['year'], errors='coerce')
df = df[df['year'] > '1880-01-01']

# Example: Constant variable
df['source'] = "NASA"

# Example: Boolean variable
df['boolean'] = np.random.choice([True, False], df.shape[0])

# Example: Mixed with base types
df['mixed'] = np.random.choice([1, "A"], df.shape[0])

# Example: Highly correlated variables
df['reclat_city'] = df['reclat'] + np.random.normal(scale=5,size=(len(df)))

# Example: Duplicate observations
duplicates_to_add = pd.DataFrame(df.iloc[0:10])
duplicates_to_add[u'name'] = duplicates_to_add[u'name'] + " copy"

df = df.append(duplicates_to_add, ignore_index=True)

In [14]:
# DataFrame conversion to dask.DataFrame
ddf = dd.from_pandas(df, npartitions=10)

### Inline report without saving object

In [15]:
dask_profiling.ProfileReport(ddf)

Number of variables,14
Number of observations,44941
Total Missing (%),3.4%
Total size in memory,4.5 MiB
Average record size in memory,105.0 B
Numeric,4
Categorical,5
Boolean,1
Date,1
Text (Unique),1
Rejected,2


### Save report to file

In [21]:
pfr = dask_profiling.ProfileReport(ddf)
pfr.to_file(os.path.join(os.path.curdir, 'example.html'))

#### Print existing ProfileReport object inline

In [22]:
pfr

Number of variables,14
Number of observations,44941
Total Missing (%),3.4%
Total size in memory,4.5 MiB
Average record size in memory,105.0 B
Numeric,4
Categorical,5
Boolean,1
Date,1
Text (Unique),1
Rejected,2
